In [1]:
! pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# january 2021
dfj = pd.read_parquet('fhv_tripdata_2021-01.parquet')
# february 2021
dff = pd.read_parquet('fhv_tripdata_2021-02.parquet')

# concatenate dataframes
df = pd.concat([dfj, dff])
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,<NA>,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,<NA>,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,<NA>,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,<NA>,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,<NA>,B00037


In [3]:
df.shape

(2191804, 7)

In [30]:
df.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                      object
DOlocationID                      object
SR_Flag                            Int32
Affiliated_base_number            object
duration                         float64
dtype: object

In [5]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropOff_datetime'] = pd.to_datetime(df['dropOff_datetime'])

In [6]:
# validate df from January with concatenated df
print(f"Number of records in Jan 2021: {dfj.shape[0]}")
print(f"Number of records in Jan 2021: {df.loc[df.pickup_datetime.dt.month == 1].shape[0]}")

Number of records in Jan 2021: 1154112
Number of records in Jan 2021: 1154112


In [7]:
del [dff, dfj]

In [8]:
# calculate trip duration
df['duration'] = df['dropOff_datetime'] - df['pickup_datetime']

In [9]:
# convert to duration in minutes
def duration_in_minutes(duration_col):
   return duration_col.total_seconds() / 60

df['duration'] = list(map(duration_in_minutes ,df.duration))

In [10]:
print(f"Average duration in Jan 2021: {round(df.loc[df.pickup_datetime.dt.month == 1]['duration'].mean(), 3)} mins.")

Average duration in Jan 2021: 19.167 mins.


In [11]:
# check missing values
df.isna().sum()

dispatching_base_num            0
pickup_datetime                 0
dropOff_datetime                0
PUlocationID              1842958
DOlocationID               314572
SR_Flag                   2191804
Affiliated_base_number        885
duration                        0
dtype: int64

In [12]:
# fill missing values with "-1"
df = df.fillna(-1)

In [13]:
df_size_old = df.shape[0]

In [14]:
# take into account only trips 1min < duration <60min
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [15]:
print(f"Number of dropped records: {df_size_old - df.shape[0]}")

Number of dropped records: 91865


In [16]:
# calculate the fraction of 'PUlocationID' NaNs
fraction_na = len(df.loc[(df.PUlocationID == -1) & (df.pickup_datetime.dt.month == 1)]) / len(df.loc[df.pickup_datetime.dt.month == 1])
fraction_na


0.8352732770722617

In [18]:
categorical = ['PUlocationID', 'DOlocationID']
target = 'duration'

# convert integers to 'object' data type
df[categorical] = df[categorical].astype(str)

In [19]:
# split in train and validation set (January, February)
df_train = df.loc[df.pickup_datetime.dt.month == 1][categorical + [target]]
df_val = df.loc[df.pickup_datetime.dt.month == 2][categorical + [target]]

In [21]:
# Convert features to a list of dictionaries
train_dicts = df_train[categorical].to_dict(orient='records')
val_dicts = df_val[categorical].to_dict(orient='records')

[{'PUlocationID': '-1.0', 'DOlocationID': '-1.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '-1.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '72.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '61.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '71.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '91.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '39.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '37.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '39.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '72.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '72.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '89.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '177.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '225.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '63.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '67.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '22.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '61.0'},
 {'PUlocationID': '-1.0', 'DOlocationID': '14.0'},
 {'PUlocationID': '-1.0', 'DO

In [24]:
# vectorize the input
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
# for the validation set only transform
X_val = dv.transform(val_dicts)

In [25]:
# number of features - dimensions
len(dv.get_feature_names())

525

In [27]:
# train and validation target column
y_train = df_train[target].values
y_val = df_val[target].values

# Linear LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)


11.014283148397356

In [29]:
# RMSE on training set
y_pred_t = lr.predict(X_train)
mean_squared_error(y_train, y_pred_t, squared=False)

10.528519107205634

In [ ]:
# RMSE on validation set
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)